# Precision

In [2]:
# imports
from helper_metrics import *
import json

## Working example

In [3]:
example_output = '''{
    "task" : "diagnosing patients",
    "desc" : "Complete the structure by giving conditions names",
    "output_structure" : {
                "Names of Conditions" : ["African Tick Bite Fever", "Choléra"]
              },
    "expanded_desc" : "a diagnostic of a patient described by a conditions names in a structured form"
  }'''

ground_truth = "Cholera"

extracted_conditions = extract_conditions(example_output)

**Exact matching**

In [31]:
matching_positions = exact_matching(ground_truth, extracted_conditions)

if matching_positions:
    print(f"Exact matches found at positions: {matching_positions}")
else:
    print("No exact matches found.")


No exact matches found.


**Fuzzy matching**

In [32]:
fuzzy_scores, matching_positions_scores = fuzzy_matching(ground_truth, extracted_conditions)

print("The fuzzy scores are: ", fuzzy_scores)

if matching_positions_scores:
    for idx, score in matching_positions_scores:
        print(f"Fuzzy match found at position {idx} with a similarity score of {score}.")
else:
    print("No fuzzy matches found above the threshold.")

The fuzzy scores are:  [13, 86]
Fuzzy match found at position 1 with a similarity score of 86.


## Pipeline

Multiple responses -> treat them as multiple cases -> Mistral will generalize and expose treatments for all of them -> Medical Assistant (not doctor) for legal reasons.

**Duplicate the prompt**

In [8]:
import json

def split_conditions_into_inputs(example_output, verbose=False):
    parsed_data = json.loads(example_output)
    conditions = parsed_data['output_structure']['Names of Conditions']

    modified_jsons = []
    for condition in conditions:
        new_data = parsed_data.copy()
        new_data['output_structure'] = {"Condition": condition}
        modified_jsons.append(json.dumps(new_data, indent=4))
    
    if verbose:
        for i, modified_json in enumerate(modified_jsons, 1):
            print(f'input_{i} = {modified_json}\n')
    
    return modified_jsons

In [9]:
multiple_prompts = split_conditions_into_inputs(example_output)
print(multiple_prompts)

['{\n    "task": "diagnosing patients",\n    "desc": "Complete the structure by giving conditions names",\n    "output_structure": {\n        "Condition": "African Tick Bite Fever"\n    },\n    "expanded_desc": "a diagnostic of a patient described by a conditions names in a structured form"\n}', '{\n    "task": "diagnosing patients",\n    "desc": "Complete the structure by giving conditions names",\n    "output_structure": {\n        "Condition": "Chol\\u00e9ra"\n    },\n    "expanded_desc": "a diagnostic of a patient described by a conditions names in a structured form"\n}']
